In [1]:
import csv
import cv2
import numpy as np

In [2]:
def process_image(img):
    return img

In [ ]:
rows = []
with open('data/internet/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        rows.append(line)
        
car_images = []
steering_angles = []

first = True

steering_correction = 0.2

for row in rows:
    # Skip header row
    if first:
        first = False
        continue
        
    for i in range(3):
        steering_center = float(row[3])

        # create adjusted steering measurements for the side camera images
        
        steering_left = steering_center + steering_correction
        steering_right = steering_center - steering_correction

        # read in images from center, left and right cameras
        path = "data/internet/IMG/" # fill in the path to your training IMG directory
        img = process_image(cv2.imread(path + row[i]))

        # add images and angles to data set
        car_images.append(img)
        
        if i == 0: 
            steering_angles.append(steering_center)
        elif i == 1: 
            steering_angles.append(steering_left)
        elif i == 2:
            steering_angles.append(steering_right)
        else:
            raise Exception('Uhhhm?')

car_images_flipped = car_images
steering_angles_flipped = steering_angles

for image, measurement in zip(car_images, steering_angles):
    car_images_flipped.append(cv2.flip(image, 1))
    steering_angles_flipped.append(-measurement)
    
X_train = np.array(car_images_flipped)
y_train = np.array(steering_angles_flipped)

print("Done!")

In [12]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Cropping2D

model = Sequential()
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((70,25), (0,0)))
model.add(Convolution2D(24,5,5,subsample=(2,2),activation="relu"))
model.add(Convolution2D(36,5,5,subsample=(2,2),activation="relu"))
model.add(Convolution2D(48,5,5,subsample=(2,2),activation="relu"))
model.add(Convolution2D(64,5,5,subsample=(2,2)))
model.add(Convolution2D(64,5,5,subsample=(2,2)))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))

model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=2)

model.save('john_drive.h5')

Train on 6428 samples, validate on 1608 samples
Epoch 1/2
6428/6428 [==============================] - 10s - loss: 1.8932 - val_loss: 0.9402
Epoch 2/2
6428/6428 [==============================] - 9s - loss: 1.9406 - val_loss: 2.4724
